In [1]:
%%spark
println("Application Id: " + spark.sparkContext.applicationId )
println("Application Name: " + spark.sparkContext.appName)

StatementMeta(sparkpoolag, 17, 2, Finished, Available)

Application Id: application_1695203745506_0001
Application Name: SitesDashboard_EnhanceSitesDataset_sparkpoolag_1695203661


## Set up variables

These will need to be changed for each enviroment

In [2]:
val storageAccountName = "mgdcag" // replace with your blob name
val storageContainerName = "sites-dashboard" //replace with your container name

// Storage path
val adls_path = f"abfss://$storageContainerName@$storageAccountName.dfs.core.windows.net"

// Sites path
val latestSitesPath = adls_path + s"/sites/latest"


spark.conf.set("mapreduce.fileoutputcommitter.marksuccessfuljobs", "false")

StatementMeta(sparkpoolag, 17, 3, Finished, Available)

storageAccountName: String = mgdcag
storageContainerName: String = sites-dashboard
adls_path: String = abfss://sites-dashboard@mgdcag.dfs.core.windows.net
latestSitesPath: String = abfss://sites-dashboard@mgdcag.dfs.core.windows.net/sites/latest


## Read the datasets into DFs (Data Frames)
This are the files created by the MGDC copy tool

In [3]:
val sitesDF =
    spark
      .read
      .format("json")
      .option("recursiveFileLookup", "false")
      .load(latestSitesPath)

StatementMeta(sparkpoolag, 17, 4, Finished, Available)

sitesDF: org.apache.spark.sql.DataFrame = [BlockAccessFromUnmanagedDevices: boolean, BlockDownloadOfAllFilesOnUnmanagedDevices: boolean ... 27 more fields]


In [4]:
val sitesCount = sitesDF.count()
println(s"The number of sites: $sitesCount")

StatementMeta(sparkpoolag, 17, 5, Finished, Available)

The number of sites: 221
sitesCount: Long = 221


# Enrich the Data

Pretty sure this is called feature engineering 

This is where we add value as SharePoint CSAs. The PG have provided the pizza base, now we need to add those toppings

## Add coloumns
### Using UDFs (User-Defined Functions):
You can define custom UDFs and use them to create new columns based on your specific logic. 

We will use this to add a boolean coloumn for OneDrive sites 

In [5]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

// returns true if site is OneDrive
// Slighty different to the example above as I was getting scalla errors
val isOneDrive = udf((siteUrl: String) => siteUrl.contains("-my.sharepoint.com"))

val sitesDFOD = sitesDF.withColumn("OneDriveSite", isOneDrive($"Url"))

StatementMeta(sparkpoolag, 17, 6, Finished, Available)

import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
isOneDrive: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$6102/1149961268@2e2e8150,BooleanType,List(Some(class[value[0]: string])),Some(class[value[0]: boolean]),None,false,true)
sitesDFOD: org.apache.spark.sql.DataFrame = [BlockAccessFromUnmanagedDevices: boolean, BlockDownloadOfAllFilesOnUnmanagedDevices: boolean ... 28 more fields]


## Enrich DF with API data

We want to call an API then append data to the DF based on the reponse.

The function below can be used to make API calls and return the repsonse as a string. This function is used futher down in the notebook

In [6]:
import org.apache.http.client.methods.HttpGet
import org.apache.http.impl.client.HttpClients
import org.apache.http.util.EntityUtils

def makeAPICall(apiUrl: String): String = {
  val httpClient = HttpClients.createDefault()
  val httpGet = new HttpGet(apiUrl)

  val response = httpClient.execute(httpGet)
  val entity = response.getEntity
  val responseJson = EntityUtils.toString(entity)

  responseJson
}

StatementMeta(sparkpoolag, 17, 7, Finished, Available)

import org.apache.http.client.methods.HttpGet
import org.apache.http.impl.client.HttpClients
import org.apache.http.util.EntityUtils
makeAPICall: (apiUrl: String)String


We will first call the last activity API to get details around file and site activity

There is a C# Azure function that contains a number of endpoints that this solution uses

In [7]:
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._
import org.json4s._
import org.json4s.jackson.JsonMethods._

// This is an Azure function that is hooked up to my tenant. Call it if you want :)
val apiUrl = "https://site-function-ag.azurewebsites.net/api/GetLastUserActivityForSites?timePeriod=D180"

// Call the API
val apiResponseJson = makeAPICall(apiUrl)

// Now you can parse apiResponseJson and process it
val jsonRDD = spark.sparkContext.parallelize(Seq(apiResponseJson))

// Load JSON data into a DataFrame without specifying the schema
val jsonDF = spark.read.json(jsonRDD)
    .withColumnRenamed("SiteId", "Id") // Rename the "SiteId" column as we use to join
    .select("Id", "lastActivityDate", "activeFileCount", "pageViewCount", "fileCount") // using select as we don't want to add all coloumns

// Join with existing dataset
val sitesDFODLA = sitesDFOD.join(jsonDF, "Id")


//sitesDFODLA.show()
display(sitesDFODLA.filter("Id == '9b88c7ff-6b3f-4df0-9f64-ec6ec52bbb54'"))


StatementMeta(sparkpoolag, 17, 8, Finished, Available)

SynapseWidget(Synapse.DataFrame, 55c25276-3377-4c4d-95d9-37e1d15daa08)


import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._
import org.json4s._
import org.json4s.jackson.JsonMethods._
apiUrl: String = https://site-function-ag.azurewebsites.net/api/GetLastUserActivityForSites?timePeriod=D180
apiResponseJson: String = [{"reportRefreshDate":"2023-09-17T00:00:00","siteId":"00ef2338-1d5b-4e42-a60a-ab44c29303ee","siteURL":"https://groverale.sharepoint.com/sites/AllTheDrives","ownerDisplayName":"AllTheDrives Owners","isDeleted":false,"lastActivityDate":null,"fileCount":2,"activeFileCount":0,"pageViewCount":0,"visitedPageCount":0,"storageUsedBytes":17683245,"storageAllocatedBytes":109521666048,"rootWebTemplate":"Group","ownerPrincipalName":"AllTheDrives@groverale.onmicrosoft.com","reportPeriod":180},{"reportRefreshDate":"2023-09-17T00:00:00","siteId":"06ee73c8-5c13-44a1-9f63-27b781646d28","siteURL":"https://groverale.sharepoint.com/sites/Adoptify2","ownerDisplayName":"Adoptify2 Owners","isDeleted":false,"lastActivityDate":"2022-10-21T00:

Next steo is to call the API for each item in the DF.

We need to do this as we want to call an API for each site and get further details for each site and enrich with specific site data

extract a list of Id values from your DataFrame and then iterate through that list to perform actions for each Id. Here's a general outline of how you can do this:

Extract a list of Id values from your DataFrame.
Iterate through the list of Id values.
For each Id, perform the desired actions.

```scala
import org.apache.spark.sql.functions._

// Assuming you have a DataFrame named "df" with a column "Id"
val idList: Array[String] = df.select("Id").distinct().collect().map(row => row.getString(0))

// Iterate through the list of Id values
for (id <- idList) {
  // Perform actions for each Id
  println(s"Processing Id: $id")

  // You can call your API or perform other actions here
}
```
This may not be the most effective, but for our usecase it makese sense. Speed is not a concern at this point.


## First Item to inliase the DFs
There may be a way to do this without using the firs item to initlaise the DFs but I'm still learning

In [8]:
import org.apache.spark.sql.functions._

// Assuming you have a DataFrame named "df" with a column "Id"
val idList: Array[String] = sitesDFODLA.select("Id").distinct().collect().map(row => row.getString(0))




StatementMeta(sparkpoolag, 17, 9, Finished, Available)

import org.apache.spark.sql.functions._
idList: Array[String] = Array(498a8cad-7133-4d55-b283-b0864d61a49a, 533fbbba-b95b-4d56-9fe8-da688e047709, d1a1ec05-7528-4768-af43-9b9593e7470f, 55c23d0d-382d-41a6-92cd-3483a8415cb2, 81beb81b-c4e3-4451-bd4c-0fbf8ab02e21, 77ff9639-8e5a-4c7a-ab29-07f8fa646bd8, 83e28332-9260-449b-8b6f-0e7f68607b7a, a36b132c-a661-4cb1-9c3f-4932e2557093, 08f7ccb0-b348-4a30-b02d-195c162d464b, bdc45e8e-c9d4-4002-bb0b-52e4215169f5, 73b8c5c4-8922-452d-80e3-1ff51e6d09cd, 803d187c-3185-4bdd-ba35-f793eccad934, b67c5ce1-fdd1-430e-a206-f274e6c4fead, d4c5bd91-4d89-4be6-bed4-60e4add22168, fe6fdc10-d612-4cc7-825d-283588127e4a, 1949f735-9790-410b-b6ad-a8f41475da3a, 42ba7d47-5bbc-4d59-94ce-bea3c495b02f, 758171a3-8598-4491-a1bb-ba81d2fdebea, ca53bf0f-e45c-4707-a9b9-1813a29c5fb7, b43fef0c-1f36-43d0-bb2c-c1b1312f71fb, a9fc9d5...


In [9]:
var lockedSite = sitesDFODLA.filter(s"Id == 'bdc45e8e-c9d4-4002-bb0b-52e4215169f5'").select("ReadLocked", "ReadOnly", "IBMode")
display(lockedSite)

StatementMeta(sparkpoolag, 17, 10, Finished, Available)

SynapseWidget(Synapse.DataFrame, 8a147ab6-65c1-4ef2-ac8c-12ba9dbde4bc)


lockedSite: org.apache.spark.sql.DataFrame = [ReadLocked: boolean, ReadOnly: boolean ... 1 more field]


In [10]:
// Pop the first item from the list to use as the schema
val firstItem = idList.head

// Perform actions for first Id
println(s"Processing Id: $firstItem")

var itemResult = sitesDFODLA.filter(s"Id == '$firstItem'").select("url", "Owner.AadObjectId").collect()
//var primaryAdminId = sitesDFODLA.filter(s"Id == '$firstItem'").select("Owner.AadObjectId")

var url = itemResult(0).getString(0)
var primaryAdminId = itemResult(0).getString(1)

val baseApiUrl = "https://site-function-ag.azurewebsites.net/api/GetAdditionalSiteInfo"

var requestUrl = s"$baseApiUrl?siteId=$firstItem&siteUrl=$url&primaryAdminId=$primaryAdminId"   

// You can call your API or perform other actions here
val apiResponseJson = makeAPICall(s"$requestUrl")

// Create a new row with ApiResponse and append it to the DataFrame
val newRowRDD = spark.sparkContext.parallelize(Seq(apiResponseJson))
val newRowDF = spark.read.json(newRowRDD)

// Check if list column had Values
if (newRowDF.select("Lists").first().get(0) == null) {
  println("No lists found")
} else {
  println("Lists found")
}

 // Expload the lists
val explodedDF = newRowDF.select(col("Lists")).withColumn("exploded_data", explode(col("Lists")))

// List DF (also mutalable)
var listDF = explodedDF.select(col("*"), col("exploded_data.*")).drop("Lists", "exploded_data")

var apiResponseDF = newRowDF.drop("lists")

display(apiResponseDF)
display(listDF)



StatementMeta(sparkpoolag, 17, 11, Finished, Available)

Processing Id: 498a8cad-7133-4d55-b283-b0864d61a49a
Lists found


SynapseWidget(Synapse.DataFrame, 7073ca12-b551-4215-8725-3629e2417f0c)

SynapseWidget(Synapse.DataFrame, 57320584-0af2-4964-9e9d-18391d027b6c)


firstItem: String = 498a8cad-7133-4d55-b283-b0864d61a49a
itemResult: Array[org.apache.spark.sql.Row] = Array([https://groverale.sharepoint.com/sites/CarTest2,e4238485-1d04-4afd-ad31-ea8cab673d94])
url: String = https://groverale.sharepoint.com/sites/CarTest2
primaryAdminId: String = e4238485-1d04-4afd-ad31-ea8cab673d94
baseApiUrl: String = https://site-function-ag.azurewebsites.net/api/GetAdditionalSiteInfo
requestUrl: String = https://site-function-ag.azurewebsites.net/api/GetAdditionalSiteInfo?siteId=498a8cad-7133-4d55-b283-b0864d61a49a&siteUrl=https://groverale.sharepoint.com/sites/CarTest2&primaryAdminId=e4238485-1d04-4afd-ad31-ea8cab673d94
apiResponseJson: String = {"siteId":"498a8cad-7133-4d55-b283-b0864d61a49a","numberOfDrives":1,"storageUsedInDrives":0,"siteHasPreservationHold":false,"storageUsedPreservationHold":0,"recycleBinSize":0,"isOrphaned":false,"isHomeSite":false,"isAccessLocked":false,"isDeleted":false,"numberOfItemsInSite":0,"lists":[{"siteId":"498a8cad-7133-4d55-b28

In [11]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Column

val remainingItems = idList.tail
//val remainingItems = idList.tail.take(5) // using 5 as we are in dev - don't want to call the API 100s

// This is another Azure function that is hooked up to my tenant. Call it if you want :)
val baseApiUrl = "https://site-function-ag.azurewebsites.net/api/GetAdditionalSiteInfo"

for (id <- remainingItems) {
    // Perform actions for each Id
    println(s"Processing Id: $id")

    var itemResult = sitesDFODLA.filter(s"Id == '$id'").select("url", "Owner.AadObjectId").collect()
    //var primaryAdminId = sitesDFODLA.filter(s"Id == '$firstItem'").select("Owner.AadObjectId")

    var url = itemResult(0).getString(0)
    var primaryAdminId = itemResult(0).getString(1)

    var requestUrl = s"$baseApiUrl?siteId=$id&siteUrl=$url&primaryAdminId=$primaryAdminId"   
    
    // You can call your API or perform other actions here
    val apiResponseJson = makeAPICall(s"$requestUrl")

    // Create a new row with ApiResponse and append it to the DataFrame
    val newRowRDD = spark.sparkContext.parallelize(Seq(apiResponseJson))
    // Create a DataFrame from the new row
    val newRowDF = spark.read.json(newRowRDD)

    // check we have lists to expand
    val nonEmptyRowsDF = newRowDF.filter(size($"Lists") > 0)

    if (nonEmptyRowsDF.count() > 0) {
        // Your code here
        println("Lists found")
        // Expload the lists
        val explodedDF = newRowDF.select(col("Lists")).withColumn("exploded_data", explode(col("Lists")))
        var listRowDF = explodedDF.select(col("*"), col("exploded_data.*")).drop("Lists", "exploded_data")

        // Add the Lists to the listDF (created in above cell)
        listDF = listDF.union(listRowDF)

        // Add the APIresponse but drop the lists
        apiResponseDF = apiResponseDF.union(newRowDF.drop("Lists"))
    }
    else {
        println("No Lists!")
    }      

}

display(apiResponseDF)
display(listDF)

StatementMeta(sparkpoolag, 17, 12, Finished, Available)

Processing Id: 533fbbba-b95b-4d56-9fe8-da688e047709
Lists found
Processing Id: d1a1ec05-7528-4768-af43-9b9593e7470f
Lists found
Processing Id: 55c23d0d-382d-41a6-92cd-3483a8415cb2
Lists found
Processing Id: 81beb81b-c4e3-4451-bd4c-0fbf8ab02e21
Lists found
Processing Id: 77ff9639-8e5a-4c7a-ab29-07f8fa646bd8
Lists found
Processing Id: 83e28332-9260-449b-8b6f-0e7f68607b7a
Lists found
Processing Id: a36b132c-a661-4cb1-9c3f-4932e2557093
Lists found
Processing Id: 08f7ccb0-b348-4a30-b02d-195c162d464b
Lists found
Processing Id: bdc45e8e-c9d4-4002-bb0b-52e4215169f5
No Lists!
Processing Id: 73b8c5c4-8922-452d-80e3-1ff51e6d09cd
Lists found
Processing Id: 803d187c-3185-4bdd-ba35-f793eccad934
Lists found
Processing Id: b67c5ce1-fdd1-430e-a206-f274e6c4fead
Lists found
Processing Id: d4c5bd91-4d89-4be6-bed4-60e4add22168
Lists found
Processing Id: fe6fdc10-d612-4cc7-825d-283588127e4a
No Lists!
Processing Id: 1949f735-9790-410b-b6ad-a8f41475da3a
Lists found
Processing Id: 42ba7d47-5bbc-4d59-94ce-bea3c

SynapseWidget(Synapse.DataFrame, 281c6b45-ffff-451f-8553-1c5bda3f8c79)

SynapseWidget(Synapse.DataFrame, 19eec17e-1a6c-4d93-be96-47f53960e526)


import org.apache.spark.sql.functions._
import org.apache.spark.sql.Column
remainingItems: Array[String] = Array(533fbbba-b95b-4d56-9fe8-da688e047709, d1a1ec05-7528-4768-af43-9b9593e7470f, 55c23d0d-382d-41a6-92cd-3483a8415cb2, 81beb81b-c4e3-4451-bd4c-0fbf8ab02e21, 77ff9639-8e5a-4c7a-ab29-07f8fa646bd8, 83e28332-9260-449b-8b6f-0e7f68607b7a, a36b132c-a661-4cb1-9c3f-4932e2557093, 08f7ccb0-b348-4a30-b02d-195c162d464b, bdc45e8e-c9d4-4002-bb0b-52e4215169f5, 73b8c5c4-8922-452d-80e3-1ff51e6d09cd, 803d187c-3185-4bdd-ba35-f793eccad934, b67c5ce1-fdd1-430e-a206-f274e6c4fead, d4c5bd91-4d89-4be6-bed4-60e4add22168, fe6fdc10-d612-4cc7-825d-283588127e4a, 1949f735-9790-410b-b6ad-a8f41475da3a, 42ba7d47-5bbc-4d59-94ce-bea3c495b02f, 758171a3-8598-4491-a1bb-ba81d2fdebea, ca53bf0f-e45c-4707-a9b9-1813a29c5fb7, b43fef0c-1f36-43d0-bb2c-c1b1312f71fb, a9fc9d5c-c8b9-4f7c-9af9-d72e5d4a51ed,...
baseApiUrl: String = https://site-function-ag.azurewebsites.net/api/GetAdditionalSiteInfo


## Join back with the main dataset
We kind of want to have one master dataset as it will make the PowerBI task easier.


In [13]:
val sitesMoreDetails = apiResponseDF
    .withColumnRenamed("SiteId", "Id")

// Join with existing dataset
val sitesDFODLAMORE = sitesDFODLA.join(sitesMoreDetails, "Id")

// We are going to join backwards as we only have 6 items in debug - Prod would use the above
//val sitesDFODLAMORE = sitesMoreDetails.join(sitesDFODLA, "Id")

display(sitesDFODLAMORE)

StatementMeta(sparkpoolag, 17, 14, Finished, Available)

SynapseWidget(Synapse.DataFrame, 5b502814-8238-4444-a757-a56b7c919f8a)


sitesMoreDetails: org.apache.spark.sql.DataFrame = [isAccessLocked: boolean, isDeleted: boolean ... 9 more fields]
sitesDFODLAMORE: org.apache.spark.sql.DataFrame = [Id: string, BlockAccessFromUnmanagedDevices: boolean ... 42 more fields]


## Big Value Data Points

This is where the real magic happens. With the data in the DF it's possible to work out previous version storage. What an insight, and we haven't even itterated every object.

In the MGDC sites data set we can make the following assumption

`PreviousVersionSize = TotalSize - TotalFileStreamSize - MetadataSize`

With the addional data we now have we can make a far better assumption. We calcucate storage used in Drive by getting the size used by call the drives. We could probably even remove the metadata size

`PreviousVersionSize = storageUsedInDrives - TotalFileStreamSize`

This is just one example of what we can do with just a few extra toppings to add to this maverlous MGDC flavoured Pizza.

We will use one of the UDFs from the start

In [14]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

// returns true if site is OneDrive
// Slighty different to the example above as I was getting scalla errors
val previousVersionSize = udf((storageUsedInDrives: BigInt, totalFileStreamSize: BigInt) => 
    storageUsedInDrives - totalFileStreamSize
)
// Assuming you have a DataFrame called "df"
// TotalSize - TotalFileStreamSize - MetadataSize - storageUsedPreservationHold
val sitesDFODLAMOREPV = sitesDFODLAMORE
    .withColumn("PreviousVersionSize", previousVersionSize($"storageUsedInDrives", $"StorageMetrics.TotalFileStreamSize"))

val pvColoumns: DataFrame = sitesDFODLAMOREPV.select("Id", "OneDriveSite", "PreviousVersionSize", "lastActivityDate")
// using truncate = flase paramer to see full urls
pvColoumns.show(20, truncate = false)

StatementMeta(sparkpoolag, 17, 15, Finished, Available)

+------------------------------------+------------+-------------------+-------------------+
|Id                                  |OneDriveSite|PreviousVersionSize|lastActivityDate   |
+------------------------------------+------------+-------------------+-------------------+
|498a8cad-7133-4d55-b283-b0864d61a49a|false       |0                  |2023-07-31T00:00:00|
|533fbbba-b95b-4d56-9fe8-da688e047709|false       |-1239601           |2018-10-10T00:00:00|
|d1a1ec05-7528-4768-af43-9b9593e7470f|false       |-408479            |null               |
|55c23d0d-382d-41a6-92cd-3483a8415cb2|false       |-8532817           |2020-06-17T00:00:00|
|81beb81b-c4e3-4451-bd4c-0fbf8ab02e21|false       |-352031            |2023-09-07T00:00:00|
|77ff9639-8e5a-4c7a-ab29-07f8fa646bd8|false       |-12634             |2023-01-27T00:00:00|
|83e28332-9260-449b-8b6f-0e7f68607b7a|false       |-292357            |null               |
|a36b132c-a661-4cb1-9c3f-4932e2557093|false       |-12110             |2022-12-1

## Write back to blob storage

We need to write our new dataset back to the blobs - We will dropit in another location

We also need to write out list DF from earlier

In [15]:
val latestSitesEnhanced = adls_path + s"/sitesenhanced/latest/"
sitesDFODLAMOREPV
    .repartition(1)
    .write
    .format("json")
    .mode("overwrite")
    .save(latestSitesEnhanced)


val latestLists= adls_path + s"/lists/latest/"
listDF
    .repartition(1)
    .write
    .format("json")
    .mode("overwrite")
    .save(latestLists)

StatementMeta(sparkpoolag, 17, 16, Finished, Available)

latestSitesEnhanced: String = abfss://sites-dashboard@mgdcag.dfs.core.windows.net/sitesenhanced/latest/
latestLists: String = abfss://sites-dashboard@mgdcag.dfs.core.windows.net/lists/latest/
